In [7]:
import copy
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from pydrake.geometry import (
    ClippingRange,
    ColorRenderCamera,
    DepthRange,
    DepthRenderCamera,
    MakeRenderEngineVtk,
    RenderCameraCore,
    RenderEngineVtkParams,
    RenderLabel,
    Role,
    StartMeshcat,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import BodyIndex
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.sensors import (
    CameraInfo,
    RgbdSensor,
)
from pydrake.visualization import (
    AddDefaultVisualization,
    ColorizeDepthImage,
    ColorizeLabelImage,
)

from manipulation.utils import RenderDiagram

In [8]:
URDF_FILE = "./lc3dc.urdf"
BASE_LINK_NAME = "base_link"

In [9]:
# helpers
def xyz_rpy_deg(xyz, rpy_deg):
    """Shorthand for defining a pose."""
    rpy_deg = np.asarray(rpy_deg)
    return RigidTransform(RollPitchYaw(rpy_deg * np.pi / 180), xyz)

In [10]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [11]:
# create diagram builder with plant and scene graph
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

In [12]:
# add robot
(robot,) = Parser(plant, f"{URDF_FILE}").AddModels(URDF_FILE)
plant.WeldFrames(
    frame_on_parent_F=plant.world_frame(),
    frame_on_child_M=plant.GetFrameByName(f"{BASE_LINK_NAME}", robot)
)

RuntimeError: /Users/yunhocho/Documents/GitHub/learn-drake/arm_teleop/./lc3dc.urdf:0: error: Failed to parse XML file: XML_ERROR_FILE_NOT_FOUND

In [ ]:
# add renderer
renderer_name = "renderer"
scene_graph.AddRenderer(
    renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))

In [ ]:
# finalize the plant
plant.Finalize()

In [ ]:
# add visualization
AddDefaultVisualization(builder=builder, meshcat=meshcat)

In [ ]:
# build the diagram
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
# publish initial visualization message with default context
simulator = Simulator(diagram)
simulator.Initialize()

In [ ]:
plant_context = plant.GetMyContextFromRoot(diagram_context)
# plant.GetActuatorNames() # -> []
plant.num_positions() # -> 4

# plant.SetPositions(plant_context, [0, 0, 0, 0])
# plant.SetPositions(plant_context, [0.5, 0.5, 0.5, 0.5])
# plant.SetPositions(plant_context, [0.2, 0., 0., 0.])
# plant.SetPositions(plant_context, [0., 0.2, 0., 0.])
# plant.SetPositions(plant_context, [0., 0., 0.785, 0.])
# plant.SetPositions(plant_context, [0., 0., 0., 0.785])
# [+ y-axis of center bar, + x-axis of center bar, rot w.r.t. +x-axis, rot w.r.t. +y-axis]